<a href="https://colab.research.google.com/github/paramjeetn/Doctor_Copilot_V1/blob/main/Notebooks/Setup_run_ChromaDb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#visualization tool for displaying long load/processing times
!pip install tqdm --quiet
#data processing
!pip install pandas --quiet
#workhorse for converting text into embeddings/vectors
!pip install sentence-transformers==2.2.2 --quiet
#data framework for LLM applications
!pip install llama-index==0.9.29  --quiet
#logging output
!pip install loguru==0.7.0 --quiet
#convenient pretty printing library
!pip install rich --quiet
#openai Tokenizer library
!pip install tiktoken --quiet

In [2]:
!curl -o Split_Data_Med.json https://raw.githubusercontent.com/paramjeetn/Doctor_Copilot_V1/main/Data/Split_Data_Med.json


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  205k  100  205k    0     0   359k      0 --:--:-- --:--:-- --:--:--  359k


In [ ]:
!pip install chromadb --quiet

In [4]:
%load_ext autoreload
%autoreload 2

#standard libraries
import sys
sys.path.append('../')

import json
import os
import time
from typing import List, Tuple
from math import ceil

#external libraries
import pandas as pd
import numpy as np
from rich import print
from rich.pretty import pprint #nifty library for pretty printing
from torch import cuda
from tqdm import tqdm

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
root_folder = '/content/'  #'../data'
data_file = 'Split_Data_Med.json'
data_path = os.path.join(root_folder, data_file)


def load_json(file_path: str) -> list[dict]:
  with open(file_path) as f:
    data = json.load(f)
  return data

# data = load_json(data_path)
# print(f'Total # of papers: {len(data)}')

In [8]:
import json
import chromadb
#change path accordingly for local and colab
path = '/content/gdrive/MyDrive/Data_Medical_Rag/ChromaDb'
client = chromadb.PersistentClient(path)

In [9]:
client.heartbeat()

1719753701513457960

In [10]:
collection = client.create_collection("Doctor_Copilot_chromaDB_Collection")


In [11]:
data = load_json(data_path)

documents_embeddings = []
metadata_list = []
ids_list = []

for doc in data:
    # Assuming you need to convert the content into a list of embeddings
    # Replace this with your actual embedding conversion logic
    documents_embeddings.append(doc["content"])  # Split the content string into floats
    metadata_list.append({"doc_id": doc["doc_id"], "doc_name": doc["doc_name"]})
    ids_list.append(doc["content_id"])

# print("Adding to collection:")
# print("Documents:", documents_embeddings)
# print("Metadata:", metadata_list)
# print("IDs:", ids_list)

collection.upsert(
    documents=documents_embeddings,
    ids=ids_list
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:08<00:00, 10.1MiB/s]


In [12]:

collection.query(
    query_texts=["Prevention of diabetes in older adults"],
    n_results=2,
)


{'ids': [['d_2_cont_2', 'd_11_cont_1']],
 'distances': [[0.8127467058934484, 0.9373169392363202]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['• Symptoms suggestive of diabetes or changes in risk (B) Diet and Physical Activity Recommendations for Adults at Risk for Type 2 Diabetes (i) Follow a Healthy Eating Pattern • Emphasize whole grains, legumes, nuts, fruits, and vegetables and minimize refined and processed foods • A variety of healthy eating patterns include: » Mediterranean-style » Low-carbohydrate » Vegetarian or plant-based » DASH (Dietary Approaches to Stop Hypertension) (ii) Get Regular Physical Activity • ≥150 min/week of moderate-intensity physical activity, such as brisk walking • May include resistance or strength training • Break up prolonged sedentary time (iii) Where to Refer: • Refer adults with overweight or obesity who are at high risk for type 2 diabetes to a recognized diabetes prevention lifestyle change program (cdc.gov/diabetes/prevent